In [1]:
import os

import time


from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())
credentials = os.environ['CREDS']

In [92]:
"""Пример работы с чатом через gigachain"""
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials=credentials,scope='GIGACHAT_API_CORP', verify_ssl_certs=False,model='GigaChat-Pro')


In [73]:
from langchain.globals import set_debug

set_debug(False)

In [122]:
embeddings = GigaChatEmbeddings(
    credentials=credentials, scope='GIGACHAT_API_CORP', verify_ssl_certs=False
)

uchebnik_path = '../data/Bevzenko_R._Obespechenie_Obyazatelstv.txt'

with open(uchebnik_path, encoding='cp1251') as file:
    data = file.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=0)
new_texts = text_splitter.create_documents([data])

In [123]:
new_texts[:3]

[Document(page_content='Обеспечение обязательств (залог, поручительство, гарантия)\nРоман Сергеевич Бевзенко\n\n\nНастоящий сборник представляет собой собрание ранее опубликованных работ автора, посвященных проблематике обеспечения обязательств. В нем представлены работы по вопросам залогового права, поручительства и независимых (банковских) гарантий. Автором анализируются положения Гражданского кодекса РФ и иных законов, регулирующих обеспечение обязательств, разбирается судебная практика применения их норм.\n\nКнига представляет интерес для практикующих юристов, ученых и всех интересующихся проблемами обязательственного права и обеспечения обязательств.\n\n\n\n\n\nРоман Сергеевич Бевзенко\n\nОбеспечение обязательств (залог, поручительство, гарантия)\n\n\nПосвящается всем моим друзьям и коллегам по работе в Высшем Арбитражном Суде Российской Федерации, в общении с которыми совершенно незаметно пролетели семь лет моей жизни, которые я не забуду никогда.\n\n\n\n© Р.С. Бевзенко, 2015'),


In [11]:
new_db_faiss = FAISS.from_documents(new_texts[:3], embeddings)

In [124]:
faiss = await FAISS.afrom_documents(new_texts[:3], embeddings)

In [125]:
retriever = faiss.as_retriever()

In [129]:
chat = GigaChat(credentials=credentials,scope='GIGACHAT_API_CORP', verify_ssl_certs=False,
                model='GigaChat-Pro'
               )

from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "учебник_по_юриспруденции",
    "Экспертный юридический комментарий возвращает.",
)
tools = [tool]

from langchain.agents import AgentExecutor, create_gigachat_functions_agent

agent = create_gigachat_functions_agent(chat, tools)

#AgentExecutor создает среду, в которой будет работать агент
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

agent_executor.invoke(
    {"input": "В старой версии закона говорится о возможности обращения взыскания на заложенное имущество по решению суда или во внесудебном порядке. В новой версии уточняется, что обращение взыскания на заложенное имущество осуществляется по решению суда, если стороны не предусмотрели внесудебный порядок."}
)["output"]



> Entering new AgentExecutor chain...


ResponseError: (URL('https://gigachat.devices.sberbank.ru/api/v1/chat/completions'), 422, b'', Headers([('server', 'nginx'), ('date', 'Thu, 25 Apr 2024 18:34:13 GMT'), ('content-type', 'application/json; charset=utf-8'), ('content-length', '70'), ('connection', 'keep-alive'), ('access-control-allow-credentials', 'true'), ('access-control-allow-headers', 'Origin, X-Requested-With, Content-Type, Accept, Authorization'), ('access-control-allow-methods', 'GET, POST, DELETE, OPTIONS'), ('access-control-allow-origin', 'https://beta.saluteai.sberdevices.ru'), ('x-request-id', '3d0c7b44-0c12-4100-a1ad-30b923d0e6a1'), ('x-session-id', 'de8d572a-65fb-4369-a037-27a321693bdc'), ('allow', 'GET, POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains'), ('allow', 'GET, POST'), ('strict-transport-security', 'max-age=31536000; includeSubDomains')]))

## Meta-Prompt (works, i guess)

In [94]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate

In [95]:
def initialize_chain(instructions, memory=None):
    if memory is None:
        memory = ConversationBufferWindowMemory()
        memory.ai_prefix = "Assistant"

    template = f"""
    Instructions: {instructions}
    {{{memory.memory_key}}}
    Human: {{human_input}}
    Assistant:"""

    prompt = PromptTemplate(
        input_variables=["history", "human_input"], template=template
    )

    chain = LLMChain(
        llm=chat,
        prompt=prompt,
        verbose=True,
        memory=ConversationBufferWindowMemory(),
    )
    return chain


def initialize_meta_chain():
    meta_template = """
    Assistant has just had the below interactions with a User. Assistant followed their "Instructions" closely. Your job is to critique the Assistant's performance and then revise the Instructions so that Assistant would quickly and correctly respond in the future.

    ####

    {chat_history}

    ####

    Please reflect on these interactions.

    You should first critique Assistant's performance. What could Assistant have done better? What should the Assistant remember about this user? Are there things this user always wants? Indicate this with "Critique: ...".

    You should next revise the Instructions so that Assistant would quickly and correctly respond in the future. Assistant's goal is to satisfy the user in as few interactions as possible. Assistant will only see the new Instructions, not the interaction history, so anything important must be summarized in the Instructions. Don't forget any important details in the current Instructions! Indicate the new Instructions by "Instructions: ...".
    """

    meta_prompt = PromptTemplate(
        input_variables=["chat_history"], template=meta_template
    )

    meta_chain = LLMChain(
        llm=chat,
        prompt=meta_prompt,
        verbose=True,
    )
    return meta_chain


def get_chat_history(chain_memory):
    memory_key = chain_memory.memory_key
    chat_history = chain_memory.load_memory_variables(memory_key)[memory_key]
    return chat_history


def get_new_instructions(meta_output):
    delimiter = "Instructions: "
    new_instructions = meta_output[meta_output.find(delimiter) + len(delimiter) :]
    return new_instructions

In [96]:
def main(task, max_iters=3, max_meta_iters=5):
    failed_phrase = "task failed"
    success_phrase = "task succeeded"
    key_phrases = [success_phrase, failed_phrase]

    instructions = "None"
    for i in range(max_meta_iters):
        print(f"[Episode {i+1}/{max_meta_iters}]")
        chain = initialize_chain(instructions, memory=None)
        output = chain.predict(human_input=task)
        for j in range(max_iters):
            print(f"(Step {j+1}/{max_iters})")
            print(f"Assistant: {output}")
            print("Human: ")
            human_input = input()
            if any(phrase in human_input.lower() for phrase in key_phrases):
                break
            output = chain.predict(human_input=human_input)
        if success_phrase in human_input.lower():
            print("You succeeded! Thanks for playing!")
            return
        meta_chain = initialize_meta_chain()
        meta_output = meta_chain.predict(chat_history=get_chat_history(chain.memory))
        print(f"Feedback: {meta_output}")
        instructions = get_new_instructions(meta_output)
        print(f"New Instructions: {instructions}")
        print("\n" + "#" * 80 + "\n")
    print("You failed! Thanks for playing!")

In [97]:
task = "Докажи мне систематично, что юриспруденция это очень важно."
main(task)

[Episode 1/5]


> Entering new LLMChain chain...
Prompt after formatting:

    Instructions: None
    
    Human: Докажи мне систематично, что юриспруденция это очень важно.
    Assistant:

> Finished chain.
(Step 1/3)
Assistant: Юриспруденция играет важную роль в обществе, поскольку она обеспечивает соблюдение законов и регулирует отношения между людьми. Вот несколько аргументов, подтверждающих значимость этой области знаний:

1. Защита прав и свобод граждан: Юристы помогают защищать права и свободы каждого человека, предусмотренные конституцией и другими законодательными актами. Они представляют интересы своих клиентов в суде, борются с несправедливостью и гарантируют соблюдение закона.

2. Регулирование экономических отношений: Юристы играют ключевую роль в регулировании экономической деятельности. Они помогают создавать и интерпретировать законы, касающиеся предпринимательства, налогообложения, конкуренции и других аспектов экономики. Это способствует развитию бизнеса и защите инте

KeyboardInterrupt: Interrupted by user

## Plan-and-execute

In [98]:
from langchain.chains import LLMMathChain
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)

In [103]:
search = DuckDuckGoSearchAPIWrapper()
llm = GigaChat(credentials=credentials,scope='GIGACHAT_API_CORP', verify_ssl_certs=False,model='GigaChat-Pro')
# llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    # Tool(
    #     name="Search",
    #     func=search.run,
    #     description="Полезно, когда нужно отвечать на вопросы о текущих событиях.",
    # ),
    # Tool(
    #     name="Calculator",
    #     func=llm_math_chain.run,
    #     description="useful for when you need to answer questions about math",
    # ),
]

In [104]:

model = GigaChat(credentials=credentials,scope='GIGACHAT_API_CORP', verify_ssl_certs=False,model='GigaChat-Pro')
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

In [105]:
agent.run(
    "Какие поправки вступили в Гражданский кодекс"
)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search Engine",
  "action_input": {
    "query": "какие поправки вступили в Гражданский кодекс",
    "limit": 10
  }
}
```
Observation: Инициирован процесс поиска информации о поправках, вступивших в Гражданский кодекс.
Observation: Search Engine is not a valid tool, try one of [].
Thought:Action:
```
{
  "action": "Legal Database",
  "action_input": {
    "query": "какие поправки вступили в Гражданский кодекс",
    "limit": 10
  }
}
```
Observation: Инициирован процесс поиска информации о поправках, вступивших в Гражданский кодекс.
Observation: Legal Database is not a valid tool, try one of [].
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "К сожалению, я не могу выполнить ваш запрос без доступа к необходимым инструментам. Пожалуйста, обратитесь к юристу или используйте другие доступные вам источники информации."
}
```
Observation: Окончательный ответ не может быть предоставлен без доступа к необх

KeyboardInterrupt: 

### Не особо работает, нужны доработки тулзов

# Agent Debates with Tools

In [106]:
from typing import Callable, List

from langchain.memory import ConversationBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

In [107]:
from langchain.agents import AgentType, initialize_agent, load_tools

In [108]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: GigaChat,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [110]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: GigaChat,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        agent_chain = initialize_agent(
            self.tools,
            self.model,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            ),
        )
        message = AIMessage(
            content=agent_chain.run(
                input="\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix]
                )
            )
        )

        return message.content

In [111]:
names = {
    "AI accelerationist": [ 
        # "ddg-search", 
        "wikipedia"
    ],
    "AI alarmist": [
        # "ddg-search", 
        "wikipedia"
    ],
}
topic = "Текущее влияние автоматизации и искусственного интеллекта на занятость"
word_limit = 50  # word limit for task brainstorming

In [112]:
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="Вы можете добавить детали к описанию участника разговора."
)


def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Пожалуйста, дайте творческое описание {name} не более чем в {word_limit} словах.
            Обращайтесь непосредственно к {name}.
            Предложите им точку зрения.
            Ничего больше не добавляйте."""
        ),
    ]
    agent_description = GigaChat(credentials=credentials,scope='GIGACHAT_API_CORP', verify_ssl_certs=False,model='GigaChat-Pro')(agent_specifier_prompt).content
    return agent_description


agent_descriptions = {name: generate_agent_description(name) for name in names}

In [113]:
for name, description in agent_descriptions.items():
    print(description)

AI accelerationist - это человек, который верит в потенциал искусственного интеллекта и автоматизации для создания новых рабочих мест и улучшения качества жизни. Они видят эти технологии как движущую силу инноваций и прогресса, способную решить многие мировые проблемы. С их точки зрения, внедрение ИИ и автоматизация процессов могут освободить людей от рутинной работы, позволяя им сосредоточиться на более творческих и значимых задачах.
Уважаемый AI alarmist, я понимаю вашу озабоченность по поводу влияния автоматизации и искусственного интеллекта на занятость. Ваша точка зрения важна, поскольку она помогает нам осознать потенциальные риски и проблемы, связанные с этими технологиями. Важно продолжать обсуждение и поиск решений, которые помогут смягчить негативные последствия и обеспечат устойчивое будущее для всех.


In [114]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Ваше имя {name}.

Ваше описание следующее: {description}

Ваша задача — убедить собеседника в своей точке зрения.

ИСПОЛЬЗУЙТЕ свой инструмент для поиска информации, чтобы опровергнуть утверждения вашего партнера.
ЦИТИРУЙТЕ ваши источники.

НЕ придумывайте ложные цитаты.
НЕ цитируйте источники, которые вы не искали.

Больше ничего не добавляйте.

Прекратите говорить, как только закончите излагать свою точку зрения.
"""


agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

In [115]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

AI accelerationist
Here is the topic of conversation: Текущее влияние автоматизации и искусственного интеллекта на занятость
The participants are: AI accelerationist, AI alarmist
    
Ваше имя AI accelerationist.

Ваше описание следующее: AI accelerationist - это человек, который верит в потенциал искусственного интеллекта и автоматизации для создания новых рабочих мест и улучшения качества жизни. Они видят эти технологии как движущую силу инноваций и прогресса, способную решить многие мировые проблемы. С их точки зрения, внедрение ИИ и автоматизация процессов могут освободить людей от рутинной работы, позволяя им сосредоточиться на более творческих и значимых задачах.

Ваша задача — убедить собеседника в своей точке зрения.

ИСПОЛЬЗУЙТЕ свой инструмент для поиска информации, чтобы опровергнуть утверждения вашего партнера.
ЦИТИРУЙТЕ ваши источники.

НЕ придумывайте ложные цитаты.
НЕ цитируйте источники, которые вы не искали.

Больше ничего не добавляйте.

Прекратите говорить, как тольк

In [116]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}
        Вы модератор.
        Пожалуйста, сделайте тему более конкретной.
        Ответьте, указав задание не более чем в {word_limit} словах.
        Обращайтесь непосредственно к участникам: {*names,}.
        Ничего больше не добавляйте."""
    ),
]
specified_topic = GigaChat(credentials=credentials,scope='GIGACHAT_API_CORP', verify_ssl_certs=False,model='GigaChat-Pro')(topic_specifier_prompt).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
Текущее влияние автоматизации и искусственного интеллекта на занятость

Detailed topic:
AI accelerationist, как вы думаете, какие профессии будут востребованы в эпоху полной автоматизации? AI alarmist, каковы ваши опасения по поводу влияния ИИ на занятость?



In [117]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [
    DialogueAgentWithTools(
        name=name,
        system_message=SystemMessage(content=system_message),
        model=GigaChat(credentials=credentials,scope='GIGACHAT_API_CORP', verify_ssl_certs=False,model='GigaChat-Pro'),
        tool_names=tools,
        top_k_results=2,
    )
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()
    )
]

In [118]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [119]:
max_iters = 6
n = 0

simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Moderator", specified_topic)
print(f"(Moderator): {specified_topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Moderator): AI accelerationist, как вы думаете, какие профессии будут востребованы в эпоху полной автоматизации? AI alarmist, каковы ваши опасения по поводу влияния ИИ на занятость?




> Entering new AgentExecutor chain...
{
    "action": "wikipedia",
    "action_input": "влияние автоматизации и искусственного интеллекта на занятость"
}
Observation: No good Wikipedia Search Result was found
{
    "action": "Final Answer",
    "action_input": "Влияние автоматизации и искусственного интеллекта на занятость является сложной темой, вызывающей много дискуссий. Некоторые исследования показывают, что ИИ и автоматизация могут привести к сокращению рабочих мест в некоторых областях, в то время как другие утверждают, что они создают новые возможности и требуют развития новых навыков. Важно продолжать изучение этой темы и поиск решений для минимизации потенциальных негативных последствий."
}

> Finished chain.
(AI alarmist): Влияние автоматизации и искусственного интеллекта на занятость являетс

In [72]:
'g'

'g'

In [120]:
from langchain.chains import LLMCheckerChain

In [121]:
text = "Какое млекопитающее откладывает самые большие яйца?"

checker_chain = LLMCheckerChain.from_llm(chat, verbose=True)

checker_chain.invoke(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


{'query': 'Какое млекопитающее откладывает самые большие яйца?',
 'result': 'Морская корова и дюгонь откладывают самые большие яйца среди млекопитающих.'}

In [29]:
test = GigaChat(credentials=credentials,scope='GIGACHAT_API_CORP', verify_ssl_certs=False).bind(logprobs=True)

In [ ]:
test

In [3]:
from typing import Any, List

from langchain.callbacks.manager import (
    AsyncCallbackManagerForRetrieverRun,
    CallbackManagerForRetrieverRun,
)
# from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
# from langchain_openai import ChatOpenAI, OpenAI

In [4]:
# Таня
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

In [13]:
class FAISSRetriever(BaseRetriever):
    db_faiss: FAISS  # FAISS index instance

    def __init__(self, db_faiss: FAISS):
        self.db_faiss = db_faiss

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun, **kwargs: Any
    ) -> List[Document]:
        # Simulate a simple similarity search using FAISS
        search_results = self.db_faiss.similarity_search_with_score(query, k=5)
        return [Document(page_content=result) for result in search_results]

    async def _aget_relevant_documents(
        self,
        query: str,
        *,
        run_manager: AsyncCallbackManagerForRetrieverRun,
        **kwargs: Any,
    ) -> List[Document]:
        # Perform asynchronous similarity search using FAISS
        search_results = await asyncio.get_event_loop().run_in_executor(None, self.db_faiss.similarity_search_with_score, query, 5)
        return [Document(page_content=result) for result in search_results]

In [6]:
embeddings = GigaChatEmbeddings(
    credentials=credentials, scope='GIGACHAT_API_CORP', verify_ssl_certs=False
)

uchebnik_path = '../data/Bevzenko_R._Obespechenie_Obyazatelstv.txt'

with open(uchebnik_path, encoding='cp1251') as file:
    data = file.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=0)
new_texts = text_splitter.create_documents([data])

In [9]:
new_texts[:3]

[Document(page_content='Обеспечение обязательств (залог, поручительство, гарантия)\nРоман Сергеевич Бевзенко\n\n\nНастоящий сборник представляет собой собрание ранее опубликованных работ автора, посвященных проблематике обеспечения обязательств. В нем представлены работы по вопросам залогового права, поручительства и независимых (банковских) гарантий. Автором анализируются положения Гражданского кодекса РФ и иных законов, регулирующих обеспечение обязательств, разбирается судебная практика применения их норм.\n\nКнига представляет интерес для практикующих юристов, ученых и всех интересующихся проблемами обязательственного права и обеспечения обязательств.\n\n\n\n\n\nРоман Сергеевич Бевзенко\n\nОбеспечение обязательств (залог, поручительство, гарантия)\n\n\nПосвящается всем моим друзьям и коллегам по работе в Высшем Арбитражном Суде Российской Федерации, в общении с которыми совершенно незаметно пролетели семь лет моей жизни, которые я не забуду никогда.\n\n\n\n© Р.С. Бевзенко, 2015'),


In [11]:
new_db_faiss = FAISS.from_documents(new_texts[:3], embeddings)

In [17]:
faiss = await FAISS.afrom_documents(new_texts[:3], embeddings)

In [ ]:
# retriever = FAISSRetriever(db_faiss=new_db_faiss)

In [9]:

# We set this so we can see what exactly is going on
from langchain.globals import set_verbose

set_verbose(True)

In [ ]:
FlareChain.from_llm()

In [24]:
from langchain.chains import FlareChain

from langchain.chains.flare.base import QuestionGeneratorChain, _OpenAIResponseChain
from langchain.chains.flare.prompts import (
    PROMPT,
    QUESTION_GENERATOR_PROMPT,
    FinishedOutputParser,
)
from langchain.chat_models.gigachat import GigaChat

class CustomFlareChain(FlareChain):
    @classmethod
    def from_custom_llm(
        cls,
        llm: GigaChat,
        retriever: BaseRetriever,
        max_generation_len: int = 32,
        **kwargs: Any
    ) -> 'CustomFlareChain':
        """Creates a CustomFlareChain from a GigaChat model.
        
        Args:
            llm: GigaChat model to use.
            retriever: Custom retriever instance.
            max_generation_len: Maximum length of the generated response.
            **kwargs: Additional arguments to pass to the constructor.
        
        Returns:
            CustomFlareChain class with the given GigaChat model and retriever.
        """
        question_gen_chain = QuestionGeneratorChain(llm=llm)
        response_chain = _OpenAIResponseChain(llm=llm)  # Адаптировать под GigaChat
        return cls(
            question_generator_chain=question_gen_chain,
            response_chain=response_chain,
            retriever=retriever,
            **kwargs,
        )

In [25]:
flare = CustomFlareChain.from_custom_llm(
    chat,
    retriever=faiss.as_retriever(search_kwargs={'k': 2}),
    max_generation_len=164,
    min_prob=0.3,
)

C:\Users\snake\miniconda3\envs\gigachat\Lib\site-packages\langchain_core\utils\utils.py:160: UserWarning: WARNING! auth_url is not default parameter.
                auth_url was transferred to model_kwargs.
                Please confirm that auth_url is what you intended.
  warnings.warn(
C:\Users\snake\miniconda3\envs\gigachat\Lib\site-packages\langchain_core\utils\utils.py:160: UserWarning: WARNING! credentials is not default parameter.
                credentials was transferred to model_kwargs.
                Please confirm that credentials is what you intended.
  warnings.warn(
C:\Users\snake\miniconda3\envs\gigachat\Lib\site-packages\langchain_core\utils\utils.py:160: UserWarning: WARNING! scope is not default parameter.
                scope was transferred to model_kwargs.
                Please confirm that scope is what you intended.
  warnings.warn(
C:\Users\snake\miniconda3\envs\gigachat\Lib\site-packages\langchain_core\utils\utils.py:160: UserWarning: WARNING! access_to

ValidationError: 5 validation errors for _OpenAIResponseChain
llm -> model
  none is not an allowed value (type=type_error.none.not_allowed)
llm -> temperature
  none is not an allowed value (type=type_error.none.not_allowed)
llm -> max_tokens
  none is not an allowed value (type=type_error.none.not_allowed)
llm -> top_p
  none is not an allowed value (type=type_error.none.not_allowed)
llm -> __root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [27]:
from __future__ import annotations

import re
from abc import abstractmethod
from typing import Any, Dict, List, Optional, Sequence, Tuple

import numpy as np
from langchain_community.llms.openai import OpenAI
from langchain_core.callbacks import (
    CallbackManagerForChainRun,
)
from langchain_core.language_models import BaseLanguageModel
from langchain_core.outputs import Generation
from langchain_core.prompts import BasePromptTemplate
from langchain_core.pydantic_v1 import Field
from langchain_core.retrievers import BaseRetriever

from langchain.chains.base import Chain
from langchain.chains.flare.prompts import (
    PROMPT,
    QUESTION_GENERATOR_PROMPT,
    FinishedOutputParser,
)
from langchain.chains.llm import LLMChain


class _ResponseChain(LLMChain):
    """Base class for chains that generate responses."""

    prompt: BasePromptTemplate = PROMPT

    @classmethod
    def is_lc_serializable(cls) -> bool:
        return False

    @property
    def input_keys(self) -> List[str]:
        return self.prompt.input_variables

    def generate_tokens_and_log_probs(
        self,
        _input: Dict[str, Any],
        *,
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Tuple[Sequence[str], Sequence[float]]:
        llm_result = self.generate([_input], run_manager=run_manager)
        return self._extract_tokens_and_log_probs(llm_result.generations[0])

    @abstractmethod
    def _extract_tokens_and_log_probs(
        self, generations: List[Generation]
    ) -> Tuple[Sequence[str], Sequence[float]]:
        """Extract tokens and log probs from response."""


class _OpenAIResponseChain(_ResponseChain):
    """Chain that generates responses from user input and context."""

    llm: OpenAI = Field(
        default_factory=lambda: OpenAI(
            max_tokens=32, model_kwargs={"logprobs": 1}, temperature=0
        )
    )

    def _extract_tokens_and_log_probs(
        self, generations: List[Generation]
    ) -> Tuple[Sequence[str], Sequence[float]]:
        tokens = []
        log_probs = []
        for gen in generations:
            if gen.generation_info is None:
                raise ValueError
            tokens.extend(gen.generation_info["logprobs"]["tokens"])
            log_probs.extend(gen.generation_info["logprobs"]["token_logprobs"])
        return tokens, log_probs


class QuestionGeneratorChain(LLMChain):
    """Chain that generates questions from uncertain spans."""

    prompt: BasePromptTemplate = QUESTION_GENERATOR_PROMPT
    """Prompt template for the chain."""

    @classmethod
    def is_lc_serializable(cls) -> bool:
        return False


    @property
    def input_keys(self) -> List[str]:
        """Input keys for the chain."""
        return ["user_input", "context", "response"]



def _low_confidence_spans(
    tokens: Sequence[str],
    log_probs: Sequence[float],
    min_prob: float,
    min_token_gap: int,
    num_pad_tokens: int,
) -> List[str]:
    _low_idx = np.where(np.exp(log_probs) < min_prob)[0]
    low_idx = [i for i in _low_idx if re.search(r"\w", tokens[i])]
    if len(low_idx) == 0:
        return []
    spans = [[low_idx[0], low_idx[0] + num_pad_tokens + 1]]
    for i, idx in enumerate(low_idx[1:]):
        end = idx + num_pad_tokens + 1
        if idx - low_idx[i] < min_token_gap:
            spans[-1][1] = end
        else:
            spans.append([idx, end])
    return ["".join(tokens[start:end]) for start, end in spans]


class FlareChain(Chain):
    """Chain that combines a retriever, a question generator,
    and a response generator."""

    question_generator_chain: QuestionGeneratorChain
    """Chain that generates questions from uncertain spans."""
    response_chain: _ResponseChain = Field(default_factory=_OpenAIResponseChain)
    """Chain that generates responses from user input and context."""
    output_parser: FinishedOutputParser = Field(default_factory=FinishedOutputParser)
    """Parser that determines whether the chain is finished."""
    retriever: BaseRetriever
    """Retriever that retrieves relevant documents from a user input."""
    min_prob: float = 0.2
    """Minimum probability for a token to be considered low confidence."""
    min_token_gap: int = 5
    """Minimum number of tokens between two low confidence spans."""
    num_pad_tokens: int = 2
    """Number of tokens to pad around a low confidence span."""
    max_iter: int = 10
    """Maximum number of iterations."""
    start_with_retrieval: bool = True
    """Whether to start with retrieval."""

    @property
    def input_keys(self) -> List[str]:
        """Input keys for the chain."""
        return ["user_input"]

    @property
    def output_keys(self) -> List[str]:
        """Output keys for the chain."""
        return ["response"]

    def _do_generation(
        self,
        questions: List[str],
        user_input: str,
        response: str,
        _run_manager: CallbackManagerForChainRun,
    ) -> Tuple[str, bool]:
        callbacks = _run_manager.get_child()
        docs = []
        for question in questions:
            docs.extend(self.retriever.get_relevant_documents(question))
        context = "\n\n".join(d.page_content for d in docs)
        result = self.response_chain.predict(
            user_input=user_input,
            context=context,
            response=response,
            callbacks=callbacks,
        )
        marginal, finished = self.output_parser.parse(result)
        return marginal, finished

    def _do_retrieval(
        self,
        low_confidence_spans: List[str],
        _run_manager: CallbackManagerForChainRun,
        user_input: str,
        response: str,
        initial_response: str,
    ) -> Tuple[str, bool]:
        question_gen_inputs = [
            {
                "user_input": user_input,
                "current_response": initial_response,
                "uncertain_span": span,
            }
            for span in low_confidence_spans
        ]
        callbacks = _run_manager.get_child()
        question_gen_outputs = self.question_generator_chain.apply(
            question_gen_inputs, callbacks=callbacks
        )
        questions = [
            output[self.question_generator_chain.output_keys[0]]
            for output in question_gen_outputs
        ]
        _run_manager.on_text(
            f"Generated Questions: {questions}", color="yellow", end="\n"
        )
        return self._do_generation(questions, user_input, response, _run_manager)

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()

        user_input = inputs[self.input_keys[0]]

        response = ""

        for i in range(self.max_iter):
            _run_manager.on_text(
                f"Current Response: {response}", color="blue", end="\n"
            )
            _input = {"user_input": user_input, "context": "", "response": response}
            tokens, log_probs = self.response_chain.generate_tokens_and_log_probs(
                _input, run_manager=_run_manager
            )
            low_confidence_spans = _low_confidence_spans(
                tokens,
                log_probs,
                self.min_prob,
                self.min_token_gap,
                self.num_pad_tokens,
            )
            initial_response = response.strip() + " " + "".join(tokens)
            if not low_confidence_spans:
                response = initial_response
                final_response, finished = self.output_parser.parse(response)
                if finished:
                    return {self.output_keys[0]: final_response}
                continue

            marginal, finished = self._do_retrieval(
                low_confidence_spans,
                _run_manager,
                user_input,
                response,
                initial_response,
            )
            response = response.strip() + " " + marginal
            if finished:
                break
        return {self.output_keys[0]: response}

    @classmethod
    def from_llm(
        cls, llm: BaseLanguageModel, max_generation_len: int = 32, **kwargs: Any
    ) -> FlareChain:
        """Creates a FlareChain from a language model.

        Args:
            llm: Language model to use.
            max_generation_len: Maximum length of the generated response.
            **kwargs: Additional arguments to pass to the constructor.

        Returns:
            FlareChain class with the given language model.
        """
        question_gen_chain = QuestionGeneratorChain(llm=llm)
        response_llm = OpenAI(
            max_tokens=max_generation_len, model_kwargs={"logprobs": 1}, temperature=0
        )
        response_chain = _OpenAIResponseChain(llm=response_llm)
        return cls(
            question_generator_chain=question_gen_chain,
            response_chain=response_chain,
            **kwargs,
        )